# L4 – Word embedding

### Resources
1. Possible [truncated svd](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html#sklearn.decomposition.TruncatedSVD) implementation of sklearn.
2. Tensorflow [tutorial](https://www.tensorflow.org/tutorials/word2vec) for word2vec and [implementation](https://github.com/tensorflow/models/tree/master/tutorials/embedding).
3. GloVe [site](https://nlp.stanford.edu/projects/glove/).
4. FastText [site](https://fasttext.cc).
5. [Gensim](https://radimrehurek.com/gensim/) library.

**Word embedding** – the collective name for a set of language modeling and feature learning techniques in natural language processing where words or phrases from the vocabulary are mapped to vectors of real numbers. Conceptually it involves a mathematical embedding from a space with one dimension per word to a continuous vector space with much higher dimensionality. Word and phrase embeddings, when used as the underlying input representation, have been shown to boost the performance in NLP tasks such as syntactic parsing, sentiment analysis and translation. Usually, problem reduces to unsupervised learning on a large corpus of text. So, learning process is based on the idea that context of word closely associated with it. For example, the context can be a document in which the word is located or adjacent words.

### 1. Data

As text corpus you may use simple english [wikipedia](https://simple.wikipedia.org/wiki/Simple_English_Wikipedia).
1. 131K articles
2. 2K common English words.

Also you can use just english [wikipedia](https://en.wikipedia.org/wiki/Wikipedia):
1. Includes 5М+ articles
2. 2M+ different tokens

#### Exercises
1. Download all wikipedia articles.
2. Make all preprocessing work that you need (e.g. remove markup).

### 2. LSA (Latent semantic analysis)
This solution uses full document as a context of word. So, we have some vocabulary $W$ and a set of documents $D$. Matrix $X$ with shape $|W| \times |D|$ at position $w, d$ stores importance of word $w$ for document $d$. If word $w$ is not found in the document $d$ than at appropriate position $X$ has 0 (obviously, matrix is sparse).

For each matrix you can find [SVD decomposition](https://en.wikipedia.org/wiki/Singular_value_decomposition)
$$X = U \Sigma V^{T} \text{, где }$$
* $U$ – orthogonal matrix $|W| \times |W|$ of left singular vectors
* $\Sigma$ – diagonal matrix $|W| \times |D|$ of singular values
* $V$ – orthogonal matrix $|D| \times |D|$  of right singular vectors

Let's suppouse that row $w$ in matrix $U\Sigma$ is a vector that represents word $w$, and row $d$ of $V$ coresponds to document $d$. In some sense we already found the embeddings of words and documents at the same time. But size of vectors are determined by documents number $|D|$.

Nevertheless you can use truncated SVD instead
$$ X \approx X_k = U_k \Sigma_k V^{T}_k \text{, where }$$
* $U_k$ – $k$ left singular vectors
* $\Sigma_k$ – diagonal matrix $|k| \times |k|$ of largest singular values
* $V_k$ – $k$ right singular vectors

Also it known that $X_k$ is the best approximation of $X$ in the term of [Frobenius norm](https://en.wikipedia.org/wiki/Matrix_norm#Frobenius_norm) for all matrices of rank $k$.

So, it is possible to obtain a compressed words' representations of size $k \ll |W|$ as rows of matrix $U_k \Sigma_k $. It just remains to understand how to calculate the original values of the matrix $X$. There is a set of approaches, here are some of them
1. Binary flag that document contains the word.
2. The number occurrences of word in document.
3. Word frequency that is better known as $tf$ (it's possible also use $\ln(1 + tf)$).
4. [$tf \cdot idf$](https://ru.wikipedia.org/wiki/TF-IDF).

#### Further readning
1. You also can read this [aticle](https://en.wikipedia.org/wiki/Latent_semantic_analysis).
2. By the way, some [modification](http://www.netflixprize.com/assets/GrandPrize2009_BPC_BellKor.pdf) of SVD decompostion won at [Netflix prize](https://en.wikipedia.org/wiki/Netflix_Prize).
3. It is easy to see that the problem of SVD decomposition is reduced to finding eigenvectors of matrix $X \cdot X^T$.
4. You can find [here](https://arxiv.org/abs/0909.4061) probabilistic algorithms of matrix decomposition.

#### Exercises
1. Find matrix $X$, using you favourite approach (maybe you need the sparse matrix class of scipy library).
2. Find word embeddings of size $k = 128$.
3. Implement k-nearest neighbor search for Euclidean norm.
4. Find 10 nearest words for the set of words: 'cat', 'loves', 'clever' and 'mandarin'.
5. Repeat experiment for cosinus as metric. What are the results?

In [1]:
import logging, gensim, bz2
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
d2word = gensim.corpora.Dictionary.load_from_text('/Users/AsyaRodigina/hse/SQuAD/SQuAD/lab-4/wiki/wiki_wordids.txt.bz2')
mm = gensim.corpora.MmCorpus('/Users/AsyaRodigina/hse/SQuAD/SQuAD/lab-4/wiki/wiki_tfidf.mm')

2018-01-31 16:06:49,096 : INFO : loaded corpus index from /Users/AsyaRodigina/hse/SQuAD/SQuAD/lab-4/wiki/wiki_tfidf.mm.index
2018-01-31 16:06:49,098 : INFO : initializing corpus reader from /Users/AsyaRodigina/hse/SQuAD/SQuAD/lab-4/wiki/wiki_tfidf.mm
2018-01-31 16:06:49,103 : INFO : accepted corpus with 84153 documents, 31952 features, 7315849 non-zero entries


In [3]:
print(d2word)
d2word.save('/tmp/d2word.dict')

2018-01-31 16:06:49,912 : INFO : saving Dictionary object under /tmp/d2word.dict, separately None
2018-01-31 16:06:49,929 : INFO : saved /tmp/d2word.dict


Dictionary(31952 unique tokens: ['aa', 'aaa', 'aachen', 'aalborg', 'aaliyah']...)


In [4]:
print(d2word.token2id)

{'aa': 8724, 'aaa': 1931, 'aachen': 15901, 'aalborg': 7087, 'aaliyah': 17815, 'aan': 25223, 'aang': 31667, 'aare': 30883, 'aargau': 14995, 'aaron': 16392, 'ab': 8727, 'aba': 14049, 'ababa': 14215, 'abad': 29076, 'abandon': 18176, 'abandoned': 3761, 'abandoning': 28967, 'abandonment': 22237, 'abba': 14880, 'abbas': 19424, 'abbasi': 28892, 'abbasid': 11335, 'abbaye': 24962, 'abbess': 17232, 'abbeville': 29831, 'abbey': 5203, 'abbot': 21554, 'abbotsford': 29814, 'abbott': 9076, 'abbottabad': 30404, 'abbreviated': 8907, 'abbreviation': 2644, 'abbreviations': 2648, 'abby': 23596, 'abbé': 29017, 'abc': 2422, 'abd': 19980, 'abdel': 7702, 'abdicate': 14736, 'abdicated': 365, 'abdicates': 655, 'abdication': 10083, 'abdomen': 19111, 'abdominal': 19103, 'abducted': 23518, 'abduction': 22338, 'abdul': 3999, 'abdulaziz': 26260, 'abdullah': 15252, 'abdur': 27542, 'abdurrahman': 19501, 'abdus': 12053, 'abe': 9960, 'abel': 2067, 'abellio': 18214, 'abercrombie': 27293, 'aberdeen': 13774, 'aberdeenshire

In [5]:
lsi = gensim.models.lsimodel.LsiModel(corpus=mm, id2word=d2word, num_topics=128)

2018-01-31 16:06:56,965 : INFO : using serial LSI version on this node
2018-01-31 16:06:56,968 : INFO : updating model with new documents
2018-01-31 16:07:05,478 : INFO : preparing a new chunk of documents
2018-01-31 16:07:06,072 : INFO : using 100 extra samples and 2 power iterations
2018-01-31 16:07:06,073 : INFO : 1st phase: constructing (31952, 228) action matrix
2018-01-31 16:07:07,052 : INFO : orthonormalizing (31952, 228) action matrix
2018-01-31 16:07:11,772 : INFO : 2nd phase: running dense svd on (228, 20000) matrix
2018-01-31 16:07:12,409 : INFO : computing the final decomposition
2018-01-31 16:07:12,410 : INFO : keeping 128 factors (discarding 21.303% of energy spectrum)
2018-01-31 16:07:12,509 : INFO : processed documents up to #20000
2018-01-31 16:07:12,520 : INFO : topic #0(16.209): 0.190*"footballer" + 0.187*"actor" + 0.176*"politician" + 0.175*"english" + 0.174*"german" + 0.169*"actress" + 0.162*"singer" + 0.153*"french" + 0.144*"writer" + 0.128*"player"
2018-01-31 16:

In [6]:
lsi.print_topics(126)

2018-01-31 16:09:54,089 : INFO : topic #0(26.019): 0.115*"movie" + 0.101*"album" + 0.097*"actor" + 0.088*"music" + 0.087*"league" + 0.084*"english" + 0.084*"january" + 0.083*"actress" + 0.081*"war" + 0.080*"released"
2018-01-31 16:09:54,092 : INFO : topic #1(17.411): 0.379*"league" + -0.322*"album" + 0.214*"football" + -0.213*"song" + 0.199*"team" + -0.189*"band" + -0.180*"released" + 0.145*"hockey" + -0.137*"music" + -0.129*"chart"
2018-01-31 16:09:54,094 : INFO : topic #2(17.070): 0.279*"league" + 0.274*"album" + 0.174*"song" + -0.165*"river" + 0.161*"released" + 0.161*"band" + 0.154*"team" + 0.148*"football" + 0.142*"played" + -0.118*"county"
2018-01-31 16:09:54,097 : INFO : topic #3(16.090): -0.198*"river" + 0.197*"actor" + 0.182*"movie" + -0.168*"league" + 0.166*"actress" + 0.159*"politician" + -0.131*"county" + 0.115*"president" + -0.110*"album" + 0.107*"writer"
2018-01-31 16:09:54,101 : INFO : topic #4(14.429): 0.236*"river" + 0.233*"county" + 0.209*"album" + -0.156*"movie" + 0.

[(0,
  '0.115*"movie" + 0.101*"album" + 0.097*"actor" + 0.088*"music" + 0.087*"league" + 0.084*"english" + 0.084*"january" + 0.083*"actress" + 0.081*"war" + 0.080*"released"'),
 (1,
  '0.379*"league" + -0.322*"album" + 0.214*"football" + -0.213*"song" + 0.199*"team" + -0.189*"band" + -0.180*"released" + 0.145*"hockey" + -0.137*"music" + -0.129*"chart"'),
 (2,
  '0.279*"league" + 0.274*"album" + 0.174*"song" + -0.165*"river" + 0.161*"released" + 0.161*"band" + 0.154*"team" + 0.148*"football" + 0.142*"played" + -0.118*"county"'),
 (3,
  '-0.198*"river" + 0.197*"actor" + 0.182*"movie" + -0.168*"league" + 0.166*"actress" + 0.159*"politician" + -0.131*"county" + 0.115*"president" + -0.110*"album" + 0.107*"writer"'),
 (4,
  '0.236*"river" + 0.233*"county" + 0.209*"album" + -0.156*"movie" + 0.144*"district" + -0.131*"game" + 0.125*"de" + 0.119*"province" + 0.118*"band" + 0.115*"song"'),
 (5,
  '0.387*"movie" + 0.305*"river" + 0.278*"county" + 0.149*"award" + 0.146*"television" + -0.142*"party

In [102]:
U, S = gensim.models.lsimodel.stochastic_svd(mm, rank=128, num_terms=len(d2word), eps=1e-6)

2018-01-31 17:05:01,800 : INFO : using 128 extra samples and 0 power iterations
2018-01-31 17:05:01,817 : INFO : 1st phase: constructing (31952, 256) action matrix
2018-01-31 17:05:10,378 : INFO : PROGRESS: at document #0
2018-01-31 17:05:18,240 : INFO : PROGRESS: at document #20000
2018-01-31 17:05:26,304 : INFO : PROGRESS: at document #40000
2018-01-31 17:05:32,451 : INFO : PROGRESS: at document #60000
2018-01-31 17:05:34,795 : INFO : PROGRESS: at document #80000
2018-01-31 17:05:35,764 : INFO : 2nd phase: constructing (256, 256) covariance matrix
2018-01-31 17:05:45,264 : INFO : PROGRESS: at document #0/84153
2018-01-31 17:05:53,234 : INFO : PROGRESS: at document #20000/84153
2018-01-31 17:06:00,013 : INFO : PROGRESS: at document #40000/84153
2018-01-31 17:06:05,540 : INFO : PROGRESS: at document #60000/84153
2018-01-31 17:06:07,342 : INFO : PROGRESS: at document #80000/84153
2018-01-31 17:06:07,594 : INFO : running dense decomposition on (256, 256) covariance matrix
2018-01-31 17:0

In [103]:
len(d2word)

31952

In [182]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(5, 0.4)
yy = np.array(d2word)
xx = U * S
model.fit(xx, yy)
dic = {}
for i in range(len(d2word)):
    dic[d2word[i]] = i
s = model.kneighbors(U[dic['cat']].reshape(1, -1), 10, return_distance=False)
print('Nearest words for cat:')
for i in range(len(s[0])):
    print(d2word[s[0, i]])
print('--------------------')
print('Nearest words for loves:')
s = model.kneighbors(U[dic['loves']].reshape(1, -1), 10, return_distance=False)
for i in range(len(s[0])):
    print(d2word[s[0, i]])
print('--------------------')
print('Nearest words for clever:')
s = model.kneighbors(U[dic['clever']].reshape(1, -1), 10, return_distance=False)
for i in range(len(s[0])):
    print(d2word[s[0, i]])
print('--------------------')
print('Nearest words for mandarin:')
s = model.kneighbors(U[dic['mandarin']].reshape(1, -1), 10, return_distance=False)
for i in range(len(s[0])):
    print(d2word[s[0, i]])
print('--------------------')

Nearest words for cat:
ouen
neurologist
mcfadden
liuzzi
canret
albright
suction
vitantonio
coincidentally
mischievous
--------------------
Nearest words for loves:
rnd
capsizes
regazzoni
jouy
chemung
faverolles
textdata
ormes
lidstrom
sparking
--------------------
Nearest words for clever:
rnd
regazzoni
faverolles
léonard
ickx
ormes
capsizes
jouy
sparking
musso
--------------------
Nearest words for mandarin:
faverolles
regazzoni
léonard
sparking
transient
jiri
verrières
rnd
withdraws
peebles
--------------------


In [183]:
model = NearestNeighbors(5, 0.4, metric='cosine')
yy = np.array(d2word)
xx = U * S
model.fit(xx, yy)
s = model.kneighbors(U[dic['cat']].reshape(1, -1), 10, return_distance=False)
print('Nearest words for cat:')
for i in range(len(s[0])):
    print(d2word[s[0, i]])
print('--------------------')
print('Nearest words for loves:')
s = model.kneighbors(U[dic['loves']].reshape(1, -1), 10, return_distance=False)
for i in range(len(s[0])):
    print(d2word[s[0, i]])
print('--------------------')
print('Nearest words for clever:')
s = model.kneighbors(U[dic['clever']].reshape(1, -1), 10, return_distance=False)
for i in range(len(s[0])):
    print(d2word[s[0, i]])
print('--------------------')
print('Nearest words for mandarin:')
s = model.kneighbors(U[dic['mandarin']].reshape(1, -1), 10, return_distance=False)
for i in range(len(s[0])):
    print(d2word[s[0, i]])
print('--------------------')

Nearest words for cat:
cat
cats
kittens
neurologist
naked
albright
lexington
espn
programme
rani
--------------------
Nearest words for loves:
loves
spiro
barbershop
cyber
seleucid
commodore
potenza
perchlorate
vast
yi
--------------------
Nearest words for clever:
clever
grace
lacked
sect
honoré
magistrates
evidently
iced
ett
turned
--------------------
Nearest words for mandarin:
mandarin
cantonese
havilland
restoring
whig
transient
alliance
walloon
marching
vendetta
--------------------


### 3. word2vec
Let's consider perhaps the most popular model of word embeddings. This is mainly due to the fact that the received vectors have interesting properties. In particular, semantically similar words have close vectors, moreover linear operation of subtraction has meaning in the resulting vector space! The context of word at this time is a window of size $2c+1$, where interested word is located in the middle.

#### Continuous bag-of-words
The core idea is very simple. There is some very long text
$$w_1, w_2, \dots, w_T.$$
Consider some word $w_t$ and its context in radius $c$, namely words
$$w_{t-c}, \dots, w_{t-1}, w_{t+1}, \dots w_{t+c}.$$
Intuition tells us that the context often really well defines the word in the middle. Then let our model restore the central word if its context is known
$$\mathcal{L} = - \frac{1}{T}\sum_{t} \log \mathbb{P}[w_t|w_{t-c}, \dots, w_{t-1}, w_{t+1}, \dots w_{t+c}].$$

For each word $w_t$ there is some vector $v_t$, but if word $w_{t+i}$ is some context than we use vector $v'_{t+i}$. In this case we can represent context of word as the following sum
$$s'_t = \sum_{c \leq i \leq c, i \neq 0} v'_{t+i}$$

And define probabilty with softmax
$$\mathbb{P}[w_t|w_{t-c}, \dots, w_{t-1}, w_{t-1}, \dots w_{t+c}] = \frac{ \exp {s'}_t^T \cdot v_t}{\sum_j \exp {s'}_t^T \cdot v_j}.$$

Note that this model is easy to present as 2-layer neural network:
* The input is a sparse vector of dimension $|W|$ which at position $w'$ has one if word $w'$ is in context.
* Further, we have matrix $V'$ of vectors $v'$.
* Further, without any non-linearity matrix $V$ of vector $v$.
* And then the softmax layer.

#### Skip-gram
Another model predicts context for some word. The design is approximately the same, we need to optimize
$$\mathcal{L} = -\frac{1}{T} \sum_t \sum_{-c \leq i \leq c, i \neq 0} \mathbb{P}[w_{t+i}|w_t].$$

The probability is approximated by the following model
$$\mathbb{P}[w_{t+i}|w_t] = \frac{ \exp {v'}_{t+i}^T \cdot v_t}{\sum_j \exp {v'}_{j}^T \cdot v_t}.$$


#### Hierarchical softmax
Creation of this mode was guided to reduce the complexity of learning. However, computing of sofmax's denominator is really expensive operation. Therefore, in practice, a few other models are used.

This method was viewed in the seminar. Detailed information can be found [here](http://www.iro.umontreal.ca/~lisa/pointeurs/hierarchical-nnlm-aistats05.pdf) and practical using [here](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf).    Briefly recall, for each word from the dictionary we define [Huffman code](https://en.wikipedia.org/wiki/Huffman_coding). More frequency words have shorter code. Then we construct the Huffman tree. In accordance with its code words are stored in leaves. In the non-leaf vertices are stored special hidden representations $h'$. 

$$\mathbb{P}[w_{t+i}|w_t] = \prod_{{h'}_{t+i}, r} \sigma\Big( \mathcal{r} \cdot {h'}^T_{t+i} v_t \Big) \text{, where}$$
* ${h'}_{t+i}$ - hidden vector of word's path $w_{t+i}$
* $\mathcal{r}$ - equals $-1$, if we turn left and $+1$ in another case
* $\sigma$ - sigmoid function

Using hierarchical softmax as approximation of softmax allows us to significantly reduce complexity of model training. 

#### Exercise
1. How can you implement skip-gram model as neural network.
2. Estimate complexity of one iteration of training the skip-gram model
  * $T$ - text size
  * $W$ - vocabulary size
  * $c$ - context radius
  * $d$ - embedding size
3. Estimate complexity of using hierarchical softmax in the skip-gram model.

#### Why sampling?
Let see at softmax funciton
$$\mathbb{P}[x_i] = \frac{\exp f(x_i|\theta)}{\sum_j \exp f(x_j|\theta)}$$
Optimizing this function is equivalent to optimizing its logarithm
$$\log \frac{\exp f(x_i|\theta)}{\sum_j \exp f(x_j|\theta)} = f(x_i|\theta) - log \sum_j \exp f(x_j|\theta)$$
Than gradient $\triangledown_{\theta} \mathbb{P}[x_i]$ is equal to
$$\triangledown_{\theta} f(x_i|\theta) - \sum_k \frac{\exp f(x_k|\theta)}{\sum_j \exp f(w_j|\theta)} \triangledown_{\theta} f(x_k|\theta)$$
Inside the sum we can find $\mathbb{P}[x_k]$ again. So, the previous expression can be written as
$$
\triangledown_{\theta} f(x_i|\theta) - \sum_k \mathbb{P}[x_k] \cdot \triangledown_{\theta} f(x_k|\theta) = 
\triangledown_{\theta} f(x_i|\theta) - \mathbb{E}_{x \sim \mathbb{P}[x]} \triangledown_{\theta} f(x|\theta)
$$
If we can sample $x \sim \mathbb{P}[x]$, then we has no sense to calculate the softmax itself. We just can train model, computing only gradient. It is possible due to the fact that we do not need the probability itself, but want to find representation of words only.

#### Negative Sampling
As the result, in practice, softmax loss functions is approximated with negative sampling, which looks like a series of logistic regressions
$$\mathcal{L} = - \frac{1}{T}\sum_{w_t}
    \sum_{-c \leq i \leq c, i \neq 0} \Big(
        \mathbb{P}[y = 1 | w_{t+i}, w_t] -
        k \cdot \mathbb{E}_{\tilde{w}_{tj} \sim \mathbb{P}[w_j|w_t]} \mathbb{P}[y = 0 | \tilde{w}_{tj}, w_t]
    \Big)
$$
where $y=1$, if  word is a context of word and $y=0$ in another case. А $\tilde{w}_{tj}$ – sampling of words that don't belong to the context. Then main trick is how we will sample words $\mathbb{P}[y | w_{t+i}, w_t]$. In paper [noise contrastive estimation](https://www.cs.helsinki.fi/u/ahyvarin/papers/Gutmann10AISTATS.pdf) some interesting approaches is suggested. But we are going to use more simple variation
$$\mathcal{L} = - \frac{1}{T}\sum_{w_t} \Big(
    \sum_{-c \leq i \leq c, i \neq 0} \big(
        \sigma({v'}_{t+i}^T v_t) +
        k \cdot \mathbb{E}_{w_j \sim \mathbb{Q}(w_j)} \sigma (-{v'}_{j}^T v_t)
    \big)
\Big),
$$
to find  $\mathbb{E}_{\tilde{w}_{tj} \sim \mathbb{P}[w_j|w_t]}$ we simply sample words $k$ times from distribution $\mathbb{Q}$, $\mathbb{P}(w_j|w_t) = \mathbb{Q}(w_j)$.

#### Additional word2vec tricks
For the best quality at the moment of learning it is proposed to use variety of tricks:
1. How to choose $\mathbb{Q}$.
2. Sampling of frequent and rare words.
3. The sampling of window size.
You are welcome to read about this in original papers.

#### Further reading
1. Briefly about both models [here](https://arxiv.org/pdf/1301.3781.pdf).
2. Hierarchical softmax and negative sampling [here](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf).

#### Exercises
1. Train skip-gram model, using negative sampling and embedding size $d$ = 256. You are free to choose size of window and number of epochs.
2. Find top-10 nearest words again. Compare Euclidean and cosinus metric. Which is better? 
3. Find 10 nearest vector for expression v(king) - v(man) + v(women). If your model is tuned well, you find representation of word queen. 
4. How could you solve the following problems?
  * Find for country its capital
  * For a word find its plural form
  * For word define antonyms

In [282]:
import logging
import os.path
import sys
import multiprocessing
from gensim.models import Word2Vec
import gensim.models
from gensim.models.word2vec import LineSentence
from gensim.corpora import WikiCorpus
from gensim.models import KeyedVectors

file = '/Users/AsyaRodigina/hse/SQuAD/SQuAD/lab-4/wiki/wiki_tfidf.mm'
direc = '/Users/AsyaRodigina/hse/SQuAD/SQuAD/lab-4/wiki'
#word_vectors = gensim.models.KeyedVectors.load_word2vec_format(file, binary=False)
model = gensim.models.Word2Vec(LineSentence(file, max_sentence_length=10000))

2018-02-03 11:31:11,157 : INFO : collecting all words and their counts
2018-02-03 11:31:11,166 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-02-03 11:31:11,249 : INFO : PROGRESS: at sentence #10000, processed 30002 words, keeping 14515 word types
2018-02-03 11:31:11,314 : INFO : PROGRESS: at sentence #20000, processed 60002 words, keeping 26481 word types
2018-02-03 11:31:11,361 : INFO : PROGRESS: at sentence #30000, processed 90002 words, keeping 37929 word types
2018-02-03 11:31:11,426 : INFO : PROGRESS: at sentence #40000, processed 120002 words, keeping 48933 word types
2018-02-03 11:31:11,477 : INFO : PROGRESS: at sentence #50000, processed 150002 words, keeping 59615 word types
2018-02-03 11:31:11,525 : INFO : PROGRESS: at sentence #60000, processed 180002 words, keeping 70150 word types
2018-02-03 11:31:11,570 : INFO : PROGRESS: at sentence #70000, processed 210002 words, keeping 80604 word types
2018-02-03 11:31:11,627 : INFO : PROGRESS: at sen

In [294]:
model.similarity(str(dic['king']), str(dic['queen']))

0.77138283738875502

In [295]:
model.most_similar(positive=[str(dic['woman']), str(dic['king'])], negative=[str(dic['man'])], topn=10)

[('83', 0.9114383459091187),
 ('6', 0.9111875891685486),
 ('7195', 0.90421062707901),
 ('8955', 0.8963980674743652),
 ('238', 0.8954718112945557),
 ('70', 0.8929046988487244),
 ('89', 0.8917174339294434),
 ('9678', 0.8897289037704468),
 ('5804', 0.8874161243438721),
 ('10566', 0.8834969997406006)]

In [304]:
model.most_similar(positive=[str(dic['russia'])], topn=10)

[('239', 0.9613697528839111),
 ('221', 0.9534899592399597),
 ('1061', 0.9531764984130859),
 ('395', 0.9471006989479065),
 ('126', 0.9463851451873779),
 ('1132', 0.941466212272644),
 ('1102', 0.9405559301376343),
 ('1183', 0.9395245313644409),
 ('460', 0.9394420385360718),
 ('120', 0.9386483430862427)]

### 4. GloVe
This model combines two ideas:
1. Factorization of the matrix, as in LSA
2. Context of word is adjacent words in the corpus, just in word2vec.

Let matrix $X$ at position $i,j$ stores how many times the word $j$ was found in the context of word $i$. This matrix is filled with our corpus of text. Then the probability that the word $j$ appears in the context of word $i$ is equal to
$$P_{ij} = \frac{X_{ij}}{\sum_k X_{ik}}$$

At the same time we want to find some function $F$ and word embeddings that 
$$F((w_i - w_j)^T w_k) = \frac{F(w_i^T w_k)}{F(w_j^T w_k)} = \frac{P_{ik}}{P_{jk}}.$$
About motivation it's better to read in the original [paper](http://nlp.stanford.edu/pubs/glove.pdf).
Possible solution may be following
$$\exp(w_i^T w_j) = P_{ij}.$$

After a series of transformations you can receive that
$$w_i^T w_j = \log X_{ij} - \log\big(\sum_k X_{ik}\big)$$

Obviously, right part doesn't depend on $j$, but the resulting model is offered to be written symmetry as
$$w_i^T w_j + b_i + b_j = \log X_{ij}$$

As loss function authors suggest to use weighted MSE
$$\mathcal{L} = \sum_{i,j = 1}^{|W|} f(X_{ij}) \Big(w_i^T w_j + b_i + b_j - \log(X_{ij}) \Big)^2,$$
where $f$ is some weight function, defined for each pair of words. It is reasonable to store $X$ as sparse matrix. It is also noteworthy that during training only **not null** values of $X$ used. Model is trained with stochastic gradient descent. As some way of regularization the authors suggest to use two different matrices $W$ и $W'$ for word and context, so model can be rewritten as
$$w_i^T {w'}_j + b_i + b_j = \log X_{ij}$$
The resulted embedding of a word is the sum of two trained views $w + w'$.

#### Some additional GloVe tricks
1. In order to understand how you can choose $f$, please, read the original paper.
2. Also you are welcome to see, how $X$ is formed in the original article.

#### Further reading
1. GloVe [site](http://nlp.stanford.edu/projects/glove/).
2. Some words about [t-SNE](http://lvdmaaten.github.io/tsne/).

#### Exercises
1. Estimate complexity of model for one iteration. Choose appropriate $c$ for you.
2. Train GloVe word embeddings ($d$=256).
3. Check that v(king) - v(man) + v(women) is approximately equal v(queen).
4. Read about [t-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding).
5. Use t-SNE to reduce the size of embeddings to 3. Make sure that the following groups of vectors are collinear (use visualization)
  * [man, woman], [Mr., Ms], [king, queen], etc
  * [CEO, company]
  * [adjective, its comparative form]

### 5. FastText
[FastText](https://arxiv.org/pdf/1607.04606.pdf) is a logical development of skip-gram model. The core feature is to present the word as the sum of its n-grams embeddings and its own embedding vector. For example, if we want to use only 3-grams, than word **fast** be reprented as n-grams **-fa**, **fas**, **ast**, **st-** and word **-fast-**. The context is encoeded in usual way, as a result similarity of word and context word is the following sum
$$s(w, c) = \sum_{n \in N_w} z^{T}_n v'_{c},$$
where $N_w$ is the set of all n-grams of word $w$ combined with word itself. The authors argue that the use of combination of 3-,4- and 5-grams helps to get better embeddings for rare and even unknown words (emdedding of word itself is null vector). For model training is proposed to use negative sampling.

You can find more information on [site](https://fasttext.cc) of FastText.

#### Exercises
1. Train FastText word embeddings ($d$=256).
2. Find some rare words in your corpus and find top-5 nearest words for them, using cosinus as metric.
3. Compare results with classic word2vec.
4. Invent some funny new word and found its top-5 nearest words again.
5. How could you compare all models. Suggest your metric (see papers for inspiration).